# Writing Scripts for Tangible Landscape

***Caitlin Haedrich, Pratikshya Regmi, Anna Petrasova and Helena Mitasova***

*Center for Geospatial Analytics at NC State University*

<img src="./img/applications.jpg" />

[Tangible Landscape](https://tangible-landscape.github.io/) is a tangible user interface for GRASS, available as an addon on [GitHub](https://github.com/tangible-landscape). Using a physical landscape model for inputs, custom workflows can be easily constructed in a Python script then the results projected back onto the landscape.

Example workflows, referred to as activites in the documentation, can be found on GitHub and documentation on how to install, configure and build your own setup with custom activities is also on GitHub.

In this notebook, we'll convert some of our workflows from the case study into Tangible Landscape activities. Tangible Landscape activities consist of two files:
1. [A python file](#python) that contains the executable analysis to run on the terrain
2. [A JSON file](#json) that contains information about the activity, what layers to display and parameters for the scanner

***

## Imports and Start GRASS

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

In [ ]:
gj.init("./nc-swine/PERMANENT");

Make a new mapset for tangible landscape:

In [ ]:
gs.run_command("g.mapset", mapset="tangible_landscape", location="./nc-swine", flags="c")

In [ ]:
!g.region -p

In [ ]:
!g.region -a vector="lagoons" res=10

<a name="python"></a>


---



## From Notebook Workflow to Executable Script

Tangible Landscape uses scripts to execute analyses on the scanned terrain. The scripts have certain formatting and parameters allowing the Tangible Landscape execute it. See the [wiki page](https://github.com/tangible-landscape/grass-tangible-landscape/wiki/Running-analyses-and-developing-workflows) for details.

The `%%file` cell magic takes the content of the cell and writes it to a file. The `%%python` magic will execute the file.

### Slope Example

In [ ]:
%%writefile slope.py
#!/usr/bin/env python3

import os
import grass.script as gs


def run_slope(scanned_elev, env, **kwargs):
    gs.run_command("r.slope.aspect", elevation=scanned_elev, slope="slope", env=env)


def main():
    env = os.environ.copy()
    env["GRASS_OVERWRITE"] = "1"
    elevation = "elevation"
    elev_resampled = "elev_resampled"
    # We use resampling to get a similar resolution as with Tangible Landscape.
    gs.run_command("g.region", raster=elevation, res=20, flags="a", env=env)
    gs.run_command("r.resamp.stats", input=elevation, output=elev_resampled, env=env)

    run_slope(scanned_elev=elev_resampled, env=env)


if __name__ == "__main__":
    main()


Now execute the script:

In [ ]:
%run slope.py

And visualize the result using the `grass.jupyter` API:

In [ ]:
map = gj.Map()
map.d_rast(map="slope")
map.show()

### Drain Path

Here is another example activity that uses pins and computes the drainage path from the pin (Question 2 from Notebook 2):

In [ ]:
%%writefile drain.py
#!/usr/bin/env python3

import os
import grass.script as gs


def run_drain(scanned_elev, scanned_calib_elev, env, points=None, **kwargs):

    if not points:
        # If there are no points, ask Tangible Landscape to generate points from
        # a change in the surface.
        points = "points"
        import analyses

        analyses.change_detection(
            scanned_calib_elev,
            scanned_elev,
            points,
            height_threshold=[10, 100],
            cells_threshold=[5, 50],
            add=True,
            max_detected=5,
            debug=True,
            env=env,
        )
    # get drainage direction raster
    gs.run_command(
        "r.watershed",
        elevation=scanned_calib_elev,
        drainage="drainage",
        env=env
    )
    
    # run drainage
    gs.run_command(
        "r.path",
        input="drainage",
        start_points=points,
        vector_path="drain",
        env=env
    )


def main():
    env = os.environ.copy()
    env["GRASS_OVERWRITE"] = "1"
    elevation = "elevation"
    elev_resampled = "elev_resampled"
    # We use resampling to get a similar resolution as with Tangible Landscape.
    gs.run_command("g.region", raster=elevation, res=20, flags="a", env=env)
    gs.run_command("r.resamp.stats", input=elevation, output=elev_resampled, env=env)

    # Create points which is the additional input needed for the process.
    points = "points"
    gs.write_command(
        "v.in.ascii",
        flags="t",
        input="-",
        output=points,
        separator="comma",
        stdin="705505.0,129735.0",
        env=env,
    )
    # Call the analysis.
    run_drain(scanned_elev=elev_resampled, scanned_calib_elev=elev_resampled, env=env, points=points)


if __name__ == "__main__":
    main()

In [ ]:
%run drain.py

In [ ]:
map = gj.Map()
map.d_rast(map="elevation")
map.d_vect(map="drain")
map.show()

<a name="python"></a>

---

## JSON Configuration File

The JSON file contains the metadata about the activity such as the title and instructions along with instructions for Tangible Landscape such as the what layers to display, scanning parameters and whether a calibration step is required. A full list of parameters is on the [wiki page](https://github.com/tangible-landscape/grass-tangible-landscape/blob/master/activities_config_documentation.md).

In [ ]:
%%file config.json
{
  "tasks": [
    {
      "layers": [
        ["d.rast", "map=slope"]
      ],
      "base": "elevation",
      "scanning_params": {
        "smooth": 10,
        "zexag": 2,
        "numscans": 1,
        "interpolate": true
      },
      "analyses": "slope.py",
      "title": "Slope and Contours",
      "author": "CSDMS Workshop 2025",
      "instructions": "Change topography and observe changes in slope."
    },
    {
      "layers": [
        ["d.rast", "map=scan_saved"],
        ["d.vect", "map=drain"]
      ],
      "base": "elevation",
      "calibrate": true,
      "scanning_params": {
        "smooth": 10,
        "zexag": 2,
        "numscans": 1,
        "interpolate": true
      },
      "analyses": "drain.py",
      "title": "Drainage Path",
      "author": "CSDMS Workshop 2025",
      "instructions": "Place pin on landscape and observe drainage path from point."
    } 
  ]
}

---

## **Try it yourself!**

Create an activity for computing the area upstream of a sample site that will be marked with a pin. This is similar to question 3 in the [Case Study notebook](./02_Case_Study.ipynb).